In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("spotify_millsongdata.csv", on_bad_lines='skip')

In [ ]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [ ]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:

df.head(10)

,artist,song,text
0,Faces,On The Beach,I don't care who is watching \r\nDon't mind w...
1,Rolling Stones,Let Me Go,You're gonna get it straight from the shoulder...
2,Gloria Gaynor,Frosty The Snowman,Thumpety thump thump \r\nThumpety thump thump...
3,Kiss,Is That You?,"Cat's droolin' on the bar stool, shake your hi..."
4,Judas Priest,Take On The World,"You got to leave your seat, you gotta get up u..."
5,Drake,Good Ones Go,"[Intro:] \r\nOh yeah, oh yeaah \r\nUh uh, oh..."
6,Lil Wayne,Blinded,"Manfred Mann's Earth Band's ""Blinded By The Li..."
7,Kari Jobe,Revelation Song,"Worthy is the Lamb who was slain \r\nHoly, ho..."
8,Carpenters,A Song For You,I've been so many places in my life and time ...
9,Michael Bolton,Let's Make A Long Story Longer,Everybody's lookin' for a happy ending \r\nBe...


In [ ]:
df['text'][0]

"I don't care who is watching  \r\nDon't mind what the surfing heads might say  \r\nAnd though I may not be no Charlie Atlas  \r\nGonna take my shirt off anyway  \r\nNow skin and bones sits easy on the high tide  \r\nAnd I'm not one for castles in the sand  \r\nI've seen a girl I once knew from the East Side  \r\nI think I spy a bottle in her hand  \r\nI think I spy a bottle in her hand  \r\nI don't need to know your birthday  \r\n'cause what you tell her I don't claim to be  \r\nThere's a place I know of called the Tick Tock  \r\nWon't you come and take a walk with me  \r\nWon't you come and take a walk with me  \r\nDidn't take too much hard work to get her  \r\nMelted like a chocolate in my arms  \r\nIf you had ever seen it better  \r\nIt never did the scenery no harm  \r\nNever did the scenery no harm  \r\n\r\n"

In [ ]:
df.shape

(5000, 3)

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

array([1.        , 0.04922652, 0.01057188, ..., 0.00291   , 0.0124097 ,
       0.02587749])

In [ ]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [ ]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if df[df['song'] == song_df].empty:
        print(f"Song '{song_df}' not found in the dataset.")
        return []  # Return an empty list if song not found

    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
recommendation('Crying Over You')

Song 'Crying Over You' not found in the dataset.


[]

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')